In [2]:
!pip install requests beautifulsoup4 feedparser pymongo rapidfuzz selenium transformers torch
import time
import requests
import json
from bs4 import BeautifulSoup
import feedparser
from pymongo import MongoClient
from datetime import datetime
import pytz
from rapidfuzz import fuzz
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from transformers import pipeline
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


/Users/dibyendu/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/dibyendu/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <F6236B89-E4CA-3330-B665-E463D537EAF3> /Users/dibyendu/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A51C8C05-245A-3989-8D3C-9A6704422CA5> /Users/dibyendu/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# MongoDB Setup
client = MongoClient('mongodb://localhost:27017/')  # Update with your MongoDB connection URI
db = client['news_database']
collection = db['news_articles']

In [6]:
# Load a summarization model and sentiment analysis model
summarizer = pipeline('summarization')
sentiment_analyzer = pipeline('sentiment-analysis')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
# Setup Selenium Chrome driver (headless mode)
def setup_driver():
    options = Options()
    options.add_argument('--headless')  # Runs Chrome in headless mode
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    service = Service('/opt/homebrew/bin/chromedriver')  # Specify path to your ChromeDriver
    driver = webdriver.Chrome(service=service, options=options)
    return driver


In [10]:
def clean_and_summarize_text(text, max_len=50, min_len=20):
    """
    Clean sensationalism and summarize the input text (title or description).
    """
    # Summarize the input text (if needed)
    summary = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)[0]['summary_text']
    
    # Bolden important points in the summary
    cleaned_summary = bolden_important_points(summary)
    
    return cleaned_summary

In [12]:
# Retrieve all articles from the collection
#articles = []
#articles = list(collection.find())
#mark_and_save_duplicate_articles();

def clean_sensationalism(article_text):
    """
    Cleans sensationalism by detecting overly emotional or sensational language 
    and rephrasing the text to focus on the facts.
    """
    
    # Break the text into sentences
    if not article_text:  # Check if article_text is None or empty
        return "Content not available or cannot be cleaned."
    
    sentences = article_text.split('. ')
    factual_sentences = []
    print(f"Sentences: {sentences}")

    # Analyze each sentence
    for sentence in sentences:
        sentiment = sentiment_analyzer(sentence)
        if sentiment[0]['label'] in ['NEGATIVE', 'POSITIVE'] and sentiment[0]['score'] > 0.7:
            # Skip overly emotional sentences or rewrite them
            print(f"Skipping sensational sentence: {sentence}")
            continue
        factual_sentences.append(sentence)

    # Join the factual sentences
    clean_text = '. '.join(factual_sentences)
    
    # Use summarization to condense the cleaned text
    summary = summarizer(clean_text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
    
    return summary


In [14]:
def convert_relative_time_to_iso(relative_time):
    """
    Convert a relative time string like '21 hours ago' to an ISO timestamp.
    """
    # Get the current time
    current_time = datetime.utcnow()

    # Use regular expressions to extract time units and values
    match = re.match(r"(\d+)\s*(\w+)\s*ago", relative_time)
    if not match:
        return None  # Return None if the format is not recognized

    amount, unit = int(match.group(1)), match.group(2)

    # Convert the time unit into a timedelta object
    if "hour" in unit:
        time_delta = timedelta(hours=amount)
    elif "minute" in unit:
        time_delta = timedelta(minutes=amount)
    elif "second" in unit:
        time_delta = timedelta(seconds=amount)
    elif "day" in unit:
        time_delta = timedelta(days=amount)
    else:
        return None  # If it's a unit we don't handle, return None

    # Subtract the timedelta from the current time
    calculated_time = current_time - time_delta

    # Convert to ISO format
    return calculated_time.isoformat() + "Z"  # Adding 'Z' to denote UTC time


In [16]:
# Fetch full article content
def fetch_full_article(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        full_text = ' '.join([p.get_text() for p in paragraphs])
        return full_text if full_text else "Full content not available"
    except Exception as e:
        print(f"Failed to fetch full content from {url}: {e}")
        return "Failed to fetch full content"

In [18]:
# Scrape Google News (Top Stories and Local News)
def scrape_google_news_1():
    driver = setup_driver()

    # Navigate to Google News
    driver.get('https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en')
    time.sleep(5)  # Wait for page to load completely

    news_data = []

    # Scraping Top Stories
    top_stories_section = driver.find_elements(By.CSS_SELECTOR, 'div.NiLAwe')[:5]  # Limiting to top 5 stories
    print(top_stories_section)
    for story in top_stories_section:
        try:
            headline = story.find_element(By.CSS_SELECTOR, 'h3').text
            url = story.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            source = story.find_element(By.CSS_SELECTOR, 'div.SVJrMe').text
            full_url = f"https://news.google.com{url[1:]}"  # Append domain for full URL

            # Fetch full news description
            full_description = fetch_full_article(full_url)

            cleaned_headline = clean_sensationalism(headline)
            cleaned_description = clean_sensationalism(full_description)
            #print("Cleaned and concise article: ")
            print(f"{cleaned_headline}\n")        
            print(f"{cleaned_description}\n")
            

            news_item = {
                "headline": headline,
                "url": full_url,
                "description": full_description,
                "source": source,
                "category": "Top Stories",
                "cleaned_headline": cleaned_headline,
                "cleaned_description": cleaned_description
            }
            print(news_item)  # Print to console
            news_data.append(news_item)
        except Exception as e:
            print(f"Error fetching top story: {e}")
    # Scraping Local News (Use similar method to scrape local news)
    local_news_section = driver.find_elements(By.CSS_SELECTOR, 'div.NiLAwe')[:5]  # Limiting to top 5 local news
    for story in local_news_section:
        try:
            headline = story.find_element(By.CSS_SELECTOR, 'h3').text
            url = story.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            source = story.find_element(By.CSS_SELECTOR, 'div.SVJrMe').text
            full_url = f"https://news.google.com{url[1:]}"  # Append domain for full URL

            # Fetch full news description
            full_description = fetch_full_article(full_url)

            news_item = {
                "headline": headline,
                "url": full_url,
                "description": full_description,
                "source": source,
                "category": "Local News"
            }
            print(news_item)  # Print to console
            news_data.append(news_item)
        except Exception as e:
            print(f"Error fetching local news: {e}")

    driver.quit()
    return news_data            


In [32]:
def scrape_article_links():
    """
    Scrape article links from Google News homepage.
    """
    driver = setup_driver()
    driver.get('https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en')

    time.sleep(5)  # Allow page to load completely

    try:
        # Find all <article> tags on the page
        articles = driver.find_elements(By.TAG_NAME, 'article')
        article_links = []

        for article in articles:
            try:
                # Extract the <a> tag within each article
                link_element = article.find_element(By.TAG_NAME, 'a')
                link = link_element.get_attribute('href')
                if link and link not in article_links:
                    article_links.append(link)
            except Exception as e:
                print(f"Error extracting link from article: {e}")

        # Print the list of links
        print(f"Found {len(article_links)} article links:")
        for link in article_links:
            print(link)

    except Exception as e:
        print(f"Error during scraping: {e}")
    finally:
        driver.quit()

# Run the scraper
if __name__ == "__main__":
    scrape_article_links()


Found 0 article links:


In [20]:
def scrape_google_news():
    driver = setup_driver()

    # Navigate to Google News
    driver.get('https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en')

    # Wait for the page to load
    wait = WebDriverWait(driver, 10)

    try:
        # Locate Top Stories
        top_stories_section = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.NiLAwe')))
        news_data = []

        for story in top_stories_section[:5]:  # Limit to top 5 stories
            try:
                headline = story.find_element(By.CSS_SELECTOR, 'h3').text
                url = story.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                source = story.find_element(By.CSS_SELECTOR, 'div.SVJrMe').text
                full_url = f"https://news.google.com{url[1:]}"  # Construct full URL

                news_item = {
                    "headline": headline,
                    "url": full_url,
                    "source": source,
                    "category": "Top Stories"
                }
                print(news_item)  # Print each story to verify
                news_data.append(news_item)
            except Exception as e:
                print(f"Error fetching story: {e}")

        driver.quit()
        return news_data

    except Exception as e:
        print(f"Error: {e}")
        driver.quit()
        return []


In [22]:
# Save news data to MongoDB
def save_to_mongodb(news_data):
    if news_data:
        collection.insert_many(news_data)
        print("News saved to MongoDB")
    else:
        print("No news to save")


In [28]:
# Main function
def main():
    #news_data = scrape_google_news()
    #save_to_mongodb(news_data)
    scrape_news_links()

if __name__ == '__main__':
    main()

Error: Message: 

